# Tableau de Répartition des Bandes de Fréquence
## Projet Mastercamp Data

Pierre Bonnin

Romain Caussignac

Antoine Combaldieu

Alice Guillou

Mehdy Michalak

Jules Sucrot

# Importation et nettoyage de la Data

In [1]:
import streamlit as st
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, MarkerCluster
import streamlit as st
import plotly.express as px

In [2]:
df = pd.read_csv("https://github.com/PRAJAM/Projet/raw/jules/Export_TER_juin2023_FIX_SafwanChendeb.csv",sep=";")


# DATA CLEAN

Enlever les valeurs nulles ou trop petites de Largeur de bande

In [3]:
df2 = df.dropna(subset=['ASS_LGBD_KHZ'])
df2 = df2[df2["ASS_LGBD_KHZ"] >1000]


Enlever les antennes qui ont été supprimées

In [ ]:
enreg_to_supp = df2[df2['MVT_CODE'] == 'SUP']['N° ENREG']
df2 = df2[~df2['N° ENREG'].isin(enreg_to_supp)]


Garder seulement les ajouts d'antenne

In [ ]:
df2  = df2[df2['MVT_CODE'] == 'ADD']

Enlever les MOD

In [ ]:
df2 = df2[df2['MVT_CODE'] != 'MOD' ]

Garder seulement les dernières modifications de chaque antenne

In [ ]:
df2['Date CAF'] = pd.to_datetime(df['Date CAF'], format='%d/%m/%Y')
df2 = df2.sort_values('Date CAF').drop_duplicates(subset = "N° ENREG", keep = 'last')

Échantillonage des données (10%)

In [5]:
sample_fraction = 0.1  
df = df.sample(frac=sample_fraction, random_state=1)

TEST

In [ ]:
df2.head(10)

# Visualisation des données

## Graph du nombre d'antennes ajoutées par mois

Création d'une nouvelle variable 'YearMonth' pour trier la création d'antenne par mois

In [6]:
df2['Date CAF'] = pd.to_datetime(df2['Date CAF'], format='%d/%m/%Y')
df2['YearMonth'] = df2['Date CAF'].dt.to_period('M')
monthly_counts = df2['YearMonth'].value_counts().sort_index().reset_index()
monthly_counts.columns = ['YearMonth', 'Number of Antennas']
monthly_counts['YearMonth'] = monthly_counts['YearMonth'].astype(str)

Plot du nombre d'ajout d'antennes par mois

In [7]:
fig = px.line(
    monthly_counts, 
    x='YearMonth', 
    y='Number of Antennas', 
    title='Évolution du nombre d\'ajout d\'antennes par mois',
    markers=True,
    labels={'YearMonth': 'Mois', 'Number of Antennas': 'Nombre d\'antennes ajoutées'}
)

fig.update_layout(
    xaxis_title='Mois',
    yaxis_title='Nombre d\'antennes ajoutées',
    xaxis_tickangle=-45
)

fig.write_html('monthly_antenna_additions.html')

## Graph du nombre d'antennes en service par mois

Création d'un nouveau DatFrame contenant le nombre d'ajout et de suppression d'antennes par mois

In [8]:
df2['Date CAF'] = pd.to_datetime(df2['Date CAF'], format='%d/%m/%Y')
df2 = df2.sort_values(by='Date CAF')
df2['YearMonth'] = df2['Date CAF'].dt.to_period('M')
adds_per_month = df2[df2['MVT_CODE'] == 'ADD'].groupby('YearMonth').size()
sups_per_month = df2[df2['MVT_CODE'] == 'SUP'].groupby('YearMonth').size()

Calcul du nombre d'antennes en service par mois

In [9]:
service_df_corrected = pd.DataFrame({'Adds': adds_per_month, 'Sups': sups_per_month}).fillna(0)

service_df_corrected['Cumulative Adds'] = service_df_corrected['Adds'].cumsum()
service_df_corrected['Cumulative Sups'] = service_df_corrected['Sups'].cumsum()

service_df_corrected['Antennas in Service'] = service_df_corrected['Cumulative Adds'] - service_df_corrected['Cumulative Sups']

service_df_corrected = service_df_corrected.reset_index()
service_df_corrected['YearMonth'] = service_df_corrected['YearMonth'].astype(str)

Plot du nombre d'antennes en service par mois

In [10]:
fig_corrected = px.line(
    service_df_corrected, 
    x='YearMonth', 
    y='Antennas in Service', 
    title='Nombre d\'antennes en service par mois',
    markers=True,
    labels={'YearMonth': 'Mois', 'Antennas in Service': 'Nombre d\'antennes en service'}
)

fig_corrected.update_layout(
    xaxis_title='Mois',
    yaxis_title='Nombre d\'antennes en service',
    xaxis_tickangle=-45
)

output_file_corrected = 'antennas_in_service.html'
fig_corrected.write_html(output_file_corrected)

## Préparation des données pour l'utilistaion des bandes de fréquences par utilisateur

In [5]:
enreg_to_supp = df2[df2['MVT_CODE'] == 'SUP']['N° ENREG']
df2 = df2[~df2['N° ENREG'].isin(enreg_to_supp)]
df2['Date CAF'] = pd.to_datetime(df2['Date CAF'], format='%d/%m/%Y')
df2 = df2.sort_values('Date CAF').drop_duplicates(subset='N° ENREG', keep='last')


df2['ASS_FRQ_KHZ'] = df2['ASS_FRQ_KHZ'].astype(int)
df2['ASS_LGBD_KHZ'] = df2['ASS_LGBD_KHZ'].astype(int)

## Histogramme de l'utilisation des largeurs de bande par utilisateur

In [10]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'browser'

df = pd.read_csv('https://github.com/PRAJAM/Projet/raw/jules/Export_TER_juin2023_FIX_SafwanChendeb.csv', sep=';')

df2 = df.dropna(subset=['ASS_LGBD_KHZ'])
df2 = df2[df2["ASS_LGBD_KHZ"] > 1000]
enreg_to_supp = df2[df2['MVT_CODE'] == 'SUP']['N° ENREG']
df2 = df2[~df2['N° ENREG'].isin(enreg_to_supp)]
df2['Date CAF'] = pd.to_datetime(df2['Date CAF'], format='%d/%m/%Y')
df2 = df2.sort_values('Date CAF').drop_duplicates(subset='N° ENREG', keep='last')
df2['ASS_FRQ_KHZ'] = df2['ASS_FRQ_KHZ'].astype(int)
df2['ASS_LGBD_KHZ'] = df2['ASS_LGBD_KHZ'].astype(int)
df2['Frequency_Band'] = df2['ASS_FRQ_KHZ'].astype(str) + '_' + df2['ASS_LGBD_KHZ'].astype(str)

frequency_counts = df2['Frequency_Band'].value_counts().reset_index()
frequency_counts.columns = ['Frequency_Band', 'Count']
frequency_counts[['ASS_FRQ_KHZ', 'ASS_LGBD_KHZ']] = frequency_counts['Frequency_Band'].str.split('_', expand=True)
frequency_counts['ASS_FRQ_KHZ'] = frequency_counts['ASS_FRQ_KHZ'].astype(int)
frequency_counts['ASS_LGBD_KHZ'] = frequency_counts['ASS_LGBD_KHZ'].astype(int)
frequency_counts['ASS_FRQ_MHZ'] = frequency_counts['ASS_FRQ_KHZ'] / 1000
frequency_counts['ASS_LGBD_MHZ'] = frequency_counts['ASS_LGBD_KHZ'] / 1000

fig = go.Figure()

colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']

for i, (_, row) in enumerate(frequency_counts.iterrows()):
    start_freq = row['ASS_FRQ_MHZ']
    end_freq = start_freq + row['ASS_LGBD_MHZ']
    hover_text = (
        f"Start Frequency: {start_freq} MHz<br>"
        f"End Frequency: {end_freq} MHz<br>"
        f"Bandwidth: {row['ASS_LGBD_MHZ']} MHz<br>"
        f"Occurrences: {row['Count']}"
    )
    color = colors[i % len(colors)]
    fig.add_trace(go.Bar(
        x=[start_freq],
        y=[row['Count']],
        width=[row['ASS_LGBD_MHZ']],
        name=f"{start_freq} MHz",
        hovertext=hover_text,
        hoverinfo="text",
        marker_color=color
    ))

fig.update_layout(
    title='Histogram of Frequency Bands Usage',
    xaxis_title='Frequency (MHz)',
    yaxis_title='Count',
    bargap=0.2,
    bargroupgap=0.1,
    xaxis=dict(tickformat=','),
    yaxis=dict(tickformat=','),
    template='plotly_white'
)

fig.show()



## Diagramme circulaire de la répartition des largeurs de bande par utilisateur

In [12]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'browser'

df = pd.read_csv('https://github.com/PRAJAM/Projet/raw/jules/Export_TER_juin2023_FIX_SafwanChendeb.csv', sep=';')

df2 = df.dropna(subset=['ASS_LGBD_KHZ'])
df2 = df2[df2["ASS_LGBD_KHZ"] > 1000]
enreg_to_supp = df2[df2['MVT_CODE'] == 'SUP']['N° ENREG']
df2 = df2[~df2['N° ENREG'].isin(enreg_to_supp)]
df2['Date CAF'] = pd.to_datetime(df2['Date CAF'], format='%d/%m/%Y')
df2 = df2.sort_values('Date CAF').drop_duplicates(subset='N° ENREG', keep='last')
df2['ASS_FRQ_KHZ'] = df2['ASS_FRQ_KHZ'].astype(int)
df2['ASS_LGBD_KHZ'] = df2['ASS_LGBD_KHZ'].astype(int)

df2['ASS_FRQ_MHZ'] = df2['ASS_FRQ_KHZ'] / 1000
df2['ASS_LGBD_MHZ'] = df2['ASS_LGBD_KHZ'] / 1000

df2['END_FRQ_MHZ'] = df2['ASS_FRQ_MHZ'] + df2['ASS_LGBD_MHZ']

df2['Year'] = df2['Date CAF'].dt.year
grouped = df2.groupby(['Year', 'Code CAF', 'BASE']).agg(
    {'ASS_FRQ_MHZ': 'count', 'ASS_LGBD_MHZ': 'sum'}
).reset_index()

grouped.columns = ['Year', 'Service', 'Affectataire', 'Num_Bands', 'Total_Bandwidth_MHz']

fig_bar = px.bar(
    grouped,
    x='Year',
    y='Total_Bandwidth_MHz',
    color='Service',
    barmode='stack',
    hover_data=['Affectataire', 'Num_Bands'],
    title='Largeur de Bande Totale par Année et par Service/Affectataire',
    color_discrete_sequence=px.colors.qualitative.Plotly  
)

fig_bar.update_layout(
    xaxis=dict(tickformat='d'),
    yaxis=dict(tickformat=','),
    template='plotly_white', 
    title_x=0.5, 
    legend=dict(
        title='Service',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)

fig_bar.show()

fig_pie = px.pie(
    grouped,
    values='Total_Bandwidth_MHz',
    names='Service',
    title='Répartition de la Largeur de Bande par Service',
    color_discrete_sequence=px.colors.qualitative.Plotly 
)

fig_pie.update_layout(
    template='plotly_white',
    title_x=0.5 
)

fig_pie.show()



## Machine learning

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import MiniBatchKMeans

numerical_cols = ['ASS_FRQ_KHZ', 'ASS_LGBD_KHZ',
                  'latitude', 'longitude']
categorical_cols = ['Code CAF']  


label_encoder = LabelEncoder()

df_encoded = df2.copy()

for col in categorical_cols:
    df_encoded[col] = label_encoder.fit_transform(df_encoded[col])

scaler = StandardScaler()

df_standardized = df_encoded.copy()

df_standardized[numerical_cols] = scaler.fit_transform(df_standardized[numerical_cols])
df_standardized[categorical_cols] = scaler.fit_transform(df_standardized[categorical_cols])
df_standardized['Date CAF'] = scaler.fit_transform(df_standardized[['Date CAF']])
df_standardized['Code CAF'] /= 3
df_standardized['Date CAF'] /= 2
df_standardized['Date CAF'] +=1.5

In [ ]:

df_standardized.boxplot(figsize=(12, 8))

Matrice de corrélation

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

correlation_matrix = df_standardized.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

KMeans

In [ ]:
from sklearn.cluster import MiniBatchKMeans

wcss = []
for i in range(1, 11):
    kmeans = MiniBatchKMeans(n_clusters=i, random_state=42)
    kmeans.fit(df_standardized)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.xlabel('Nombre de Clusters')
plt.ylabel('WCSS (Inertia)')
plt.title('Méthode du Coude pour K-Means')
plt.show()


In [ ]:
df_standardized.head()
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(df_standardized)

In [ ]:
df_standardized['cluster'] = kmeans.labels_
df_standardized.boxplot(by='cluster', figsize=(12, 8))


In [ ]:
import plotly.express as px
fig = px.scatter(df2, x='Date CAF', y='ASS_FRQ_KHZ', color="Code CAF")

fig.show()

In [ ]:

import plotly.express as px
fig = px.scatter(df2, x='Date CAF', y='ASS_FRQ_KHZ', color=kmeans.labels_)


fig.show()

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

X = df_standardized[['ASS_FRQ_KHZ', 'ASS_LGBD_KHZ', 'longitude', 'latitude', 'Date CAF']]
y = df2['Code CAF']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred, average='weighted')
print('F1 Score:', f1)

from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', confusion)



Random Forest

In [ ]:


from sklearn.ensemble import RandomForestRegressor

X = df_standardized[['ASS_LGBD_KHZ', 'longitude', 'latitude', 'Date CAF']]
y = df2['ASS_FRQ_KHZ']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

from matplotlib import pyplot as plt
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

import matplotlib.pyplot as plt
plt.scatter(y_test, y_pred)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.show()
